# Lab 6-2: Softmax Classification with Real Data

## Imports

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility
torch.manual_seed(1)

## Data

In [3]:
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)

In [4]:
x_train = torch.FloatTensor(xy[:, 0:-1])
y_train = torch.LongTensor(xy[:, [-1]]).squeeze()

In [5]:
print(x_train.shape) # x_train shape
print(x_train[:5])   # first 5 rows

torch.Size([101, 16])
tensor([[1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 4., 1., 0., 1.],
        [0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0.],
        [1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 1., 0., 1.]])


In [6]:
print(y_train.shape) # y_train shape
print(y_train[:5])   # first 5 elements

torch.Size([101])
tensor([0, 0, 3, 0, 0])


In [7]:
nb_classes = 7
y_one_hot = torch.zeros((len(y_train), nb_classes))
y_one_hot = y_one_hot.scatter(1, y_train.unsqueeze(1), 1)

## Training with `F.cross_entropy`

In [8]:
W = torch.zeros((16, 7), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost calc (2)
    z = x_train.matmul(W) + b # or .mm or @
    cost = F.cross_entropy(z, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.945910
Epoch  100/1000 Cost: 0.471836
Epoch  200/1000 Cost: 0.326327
Epoch  300/1000 Cost: 0.257839
Epoch  400/1000 Cost: 0.215762
Epoch  500/1000 Cost: 0.186603
Epoch  600/1000 Cost: 0.164898
Epoch  700/1000 Cost: 0.147955
Epoch  800/1000 Cost: 0.134279
Epoch  900/1000 Cost: 0.122962
Epoch 1000/1000 Cost: 0.113422


## High-level Implementation with `nn.Module`

In [9]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(16, 7)
    def forward(self, x):
        return self.linear(x)

In [14]:
model = SoftmaxClassifierModel()

In [18]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))
        check_accuracy(model, x_train, y_train)

Epoch    0/1000 Cost: 0.054918
accuracy:  1.0
Epoch  100/1000 Cost: 0.052531
accuracy:  1.0
Epoch  200/1000 Cost: 0.050344
accuracy:  1.0
Epoch  300/1000 Cost: 0.048333
accuracy:  1.0
Epoch  400/1000 Cost: 0.046478
accuracy:  1.0
Epoch  500/1000 Cost: 0.044761
accuracy:  1.0
Epoch  600/1000 Cost: 0.043168
accuracy:  1.0
Epoch  700/1000 Cost: 0.041685
accuracy:  1.0
Epoch  800/1000 Cost: 0.040302
accuracy:  1.0
Epoch  900/1000 Cost: 0.039009
accuracy:  1.0
Epoch 1000/1000 Cost: 0.037797
accuracy:  1.0


<div class="alert alert-warning">
    Should I display how many it got correct in the training set?
</div>

## Check correction rate

In [23]:
def check_accuracy(model, x, y):
    params = list(model.parameters())
    W= params[0]
    b = params[1]
    #print("W: {}\n b: {}".format(W, b))

    total = y.shape[0]
    correct = 0

    for i in range(total):
        ansi=F.softmax(model(x[i]),dim=0)
        if (ansi.argmax().item() == y[i].item()):
            correct += 1
        else:
            print("wrong prediction at index: {}, wrong catagory: {} possibility: {:.3f}, while correct catagory: {} possibility:{:.3f}"
                .format(i,ansi.argmax().item(),ansi.max().item(),
                        y[i].item(),ansi[y[i].item()].item()))
    print("accuracy: {} ({}/{})".format(correct/total, correct, total))

Now, let's train on first half data, and test it on second half.

In [27]:
x_train = torch.FloatTensor(xy[:50, 0:-1])
y_train = torch.LongTensor(xy[:50, [-1]]).squeeze()
model = SoftmaxClassifierModel()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 200
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 20 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))
        check_accuracy(model, torch.FloatTensor(xy[50:, 0:-1]), torch.LongTensor(xy[50:, [-1]]).squeeze())

Epoch    0/200 Cost: 1.910806
wrong prediction at index: 1, wrong catagory: 0 possibility: 0.404, while correct catagory: 5 possibility:0.232
wrong prediction at index: 2, wrong catagory: 0 possibility: 0.316, while correct catagory: 4 possibility:0.025
wrong prediction at index: 3, wrong catagory: 0 possibility: 0.523, while correct catagory: 6 possibility:0.093
wrong prediction at index: 6, wrong catagory: 5 possibility: 0.231, while correct catagory: 1 possibility:0.175
wrong prediction at index: 7, wrong catagory: 0 possibility: 0.305, while correct catagory: 1 possibility:0.143
wrong prediction at index: 8, wrong catagory: 0 possibility: 0.257, while correct catagory: 1 possibility:0.149
wrong prediction at index: 9, wrong catagory: 0 possibility: 0.277, while correct catagory: 1 possibility:0.137
wrong prediction at index: 10, wrong catagory: 6 possibility: 0.224, while correct catagory: 3 possibility:0.136
wrong prediction at index: 11, wrong catagory: 6 possibility: 0.231, whil